In [1]:
import pandas as pd
import sasoptpy as so
import requests
import os
import time
import numpy as np
import subprocess 

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def get_data():
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    fpl_data = r.json()
    element_data = pd.DataFrame(fpl_data['elements'])
    team_data = pd.DataFrame(fpl_data['teams'])
    elements_team = pd.merge(element_data, team_data, left_on='team', right_on='id')
    review_data = pd.read_csv('../data/fplreview.csv')
    merged_data = pd.merge(elements_team, review_data, left_on=['name', 'web_name'], right_on=['Team', 'Name'])
    merged_data.set_index(['id_x'], inplace=True)
    next_gw = int(review_data.keys()[6].split('_')[0])
    type_data = pd.DataFrame(fpl_data['element_types']).set_index(['id'])
    
    return {'merged_data': merged_data, 'team_data': team_data, 'type_data': type_data, 'next_gw': next_gw}

In [4]:
def solve_single_period_fpl(budget):
    data = get_data()
    
    merged_data = data['merged_data']
    team_data = data['team_data']
    type_data = data['type_data']
    next_gw = data['next_gw']
    
    players = merged_data.index.tolist()
    element_types = type_data.index.tolist()
    teams = team_data['name'].tolist()
    
    model = so.Model(name='single_period')
    
    # Variables
    squad = model.add_variables(players, name='squad', vartype=so.binary)
    lineup = model.add_variables(players, name='lineup', vartype=so.binary)
    captain = model.add_variables(players, name='captain', vartype=so.binary)
    vicecap = model.add_variables(players, name='vicecap', vartype=so.binary)
    
    # Constraints
    squad_count = so.expr_sum(squad[p] for p in players)
    model.add_constraint(squad_count == 15, name='squad_count');
    model.add_constraint(so.expr_sum(lineup[p] for p in players) == 11, name='lineup_count');
    model.add_constraint(so.expr_sum(captain[p] for p in players) == 1, name='captain_count');
    model.add_constraint(so.expr_sum(vicecap[p] for p in players) == 1, name='vicecap_count');
    
    # lineup has to have less players than squad
    model.add_constraints((lineup[p] <= squad[p] for p in players), name='lineup_squad_rel'); 

    # captain has to be in lineup
    model.add_constraints((captain[p] <= lineup[p] for p in players), name='captain_lineup_rel'); 

    # vice captain has to be in lineup
    model.add_constraints((vicecap[p] <= lineup[p] for p in players), name='vicecap_lineup_rel'); 

    # captain and vice captain cannot be the same player
    model.add_constraints((captain[p] + vicecap[p] <= 1 for p in players), name='captain_vicecap_rel');
    
    lineup_type_count = {t: so.expr_sum(lineup[p] for p in players if merged_data.loc[p, 'element_type'] == t)
                     for t in element_types}
    
    squad_type_count = {t: so.expr_sum(squad[p] for p in players if merged_data.loc[p, 'element_type'] == t)
                     for t in element_types}
    
    model.add_constraints((lineup_type_count[t] == [type_data.loc[t, 'squad_min_play'], type_data.loc[t, 'squad_max_play']]
                       for t in element_types), name='valid_formation');
    model.add_constraints((squad_type_count[t] == type_data.loc[t, 'squad_select'] for t in element_types),
                      name='valid_squad');
    
    price = so.expr_sum(merged_data.loc[p, 'now_cost'] / 10 * squad[p] for p in players)
    model.add_constraint(price <= budget, name='satisfies_budget');
    model.add_constraints((so.expr_sum(squad[p] for p in players if merged_data.loc[p, 'name'] == t) <= 3 for t in teams), 
                     name='team_limit');
    
    # Objective
    total_points = so.expr_sum(merged_data.loc[p, f'{next_gw}_Pts'] * (lineup[p] + captain[p] + 0.1*vicecap[p])
                           for p in players)
    model.set_objective(-total_points, sense='N', name='total_xp');
    
    # Solution
    model.export_mps(filename=f'single_period_{budget}.mps')
    command = f'cbc single_period_{budget}.mps solve solu sp_solution_{budget}.txt'
    subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    for v in model.get_variables():
        v.set_value(0)
        
    with open(f'sp_solution_{budget}.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))
            
    picks = []
    for p in players:
        if squad[p].get_value() > 0.5:
            lp = merged_data.loc[p]
            is_lineup = 1 if lineup[p].get_value() > 0.5 else 0
            is_captain = 1 if captain[p].get_value() > 0.5 else 0
            is_vicecap = 1 if vicecap[p].get_value() > 0.5 else 0
            position = type_data.loc[lp['element_type'], 'singular_name_short']
            picks.append([
                lp['web_name'], position, lp['element_type'], lp['name'], lp['now_cost']/10,
                round(lp[f'{next_gw}_Pts'], 2), is_lineup, is_captain, is_vicecap
            ])
        
    picks_df = pd.DataFrame(picks, columns = ['name', 'position', 'type', 'team', 'price', 'xP', 'lineup',
                                          'captain', 'vice captain']).sort_values(by=['lineup', 'type', 'xP'],
                                           ascending=[False, True, True])
    
    total_xp = so.expr_sum((lineup[p] + captain[p]) * merged_data.loc[p, f'{next_gw}_Pts'] for p in players).get_value()
    
    print(f'Expected points for GW{next_gw}: {np.round(total_xp, 2)}')
    
    return picks_df

In [5]:
solve_single_period_fpl(100)

NOTE: Initialized model single_period.
Expected points for GW3: 64.63


,name,position,type,team,price,xP,lineup,captain,vice captain
6,Flekken,GKP,1,Brentford,4.5,3.74,1,0,0
3,Konsa,DEF,2,Aston Villa,4.5,3.58,1,0,0
8,Gusto,DEF,2,Chelsea,5.0,3.70,1,0,0
0,Gabriel,DEF,2,Arsenal,6.0,4.26,1,0,0
13,Gibbs-White,MID,3,Nott'm Forest,6.5,5.02,1,0,0
7,Mbeumo,MID,3,Brentford,7.0,6.01,1,0,0
11,M.Salah,MID,3,Liverpool,12.6,6.07,1,0,0
2,Saka,MID,3,Arsenal,10.1,6.79,1,0,1
9,Palmer,MID,3,Chelsea,10.6,7.04,1,1,0
1,Havertz,FWD,4,Arsenal,8.1,5.50,1,0,0
